# Les eit generelt endepunkt i REST
Først litt generell kode for å kome i gang:

In [1]:
import requests
import pandas as pd
import os

parametre = {'per_page': '100'}
hode = {'Authorization': 'Bearer ' + os.environ["tokenCanvas"]}


Så definerer eg url (og dersom det er spesielle parametre kan eg redefinere dei her):

In [12]:
url = "https://hvl.instructure.com:443/api/v1/eportfolios/39986/pages"
# parametre = {"per_page": 100}

Først gjer eg eit "enkelt" kall, berre for å sjekke korleis data ser ut (men no får eg maksimalt 100 svar):

In [10]:
respons = requests.get(url, headers=hodeCanvas, params=parametreCanvas)
if 200 <= respons.status_code < 300:
    data = respons.json()
else:
    print("Feil: ", respons.status_code)

I den grad det er aktuelt kan eg så sjå på strukturen på datasettet (eller lese dokumentasjonen ...) for å bestemme meg om eg skal lage ei dataramme, eller om eg berre skal hente ut data som ei liste:

In [ ]:
data[0]

Dersom eg får ein mistanke om at det er meir enn 100 svar kan eg køyre denne rutina som henter alle:

In [18]:
def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')
    return link_header_dict


dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(url, headers=hodeCanvas, params=parametreCanvas)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
        df = pd.DataFrame(data)
        dr_liste.append(df)
# og etter at eg er ferdig å lese inn slår eg dei saman:
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)


Og så kan eg i ro og mak sjå på data:

In [ ]:
alledata